In [25]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
import pandas as pd

file_path = "Heart_Disease_Prediction.csv" # Make sure to change this
df_LDA = pd.read_csv(file_path).drop(["index"], axis = 1)
target_variable = df_LDA['Heart Disease']
df_LDA = df_LDA.drop(['BP', 'Cholesterol', 'FBS over 120', 'EKG results','Heart Disease'], axis = 1)

n_col = len(df_LDA)
n_feat = len(df_LDA.columns)

X = df_LDA
y = target_variable
print(X)
print(y)

     Age  Sex  Chest pain type  Max HR  Exercise angina  ST depression  \
0     70    1                4     109                0            2.4   
1     67    0                3     160                0            1.6   
2     57    1                2     141                0            0.3   
3     64    1                4     105                1            0.2   
4     74    0                2     121                1            0.2   
..   ...  ...              ...     ...              ...            ...   
265   52    1                3     162                0            0.5   
266   44    1                2     173                0            0.0   
267   56    0                2     153                0            1.3   
268   57    1                4     148                0            0.4   
269   67    1                4     108                1            1.5   

     Slope of ST  Number of vessels fluro  Thallium  
0              2                        3         3  
1  

In [26]:
array = []
for x in range(len(y)):
    if(y[x] == "Absence"):
        array.append(0)
    else:
        array.append(1)
        
y_binary = array

In [42]:
best_score = -1
best_splits = 0
best_repeats = 0
best_random = 0

for j in range(10, 40, 10):
    for k in range(1, 11):
            for z in range(1, 11):
                model = LinearDiscriminantAnalysis()
                cv = RepeatedStratifiedKFold(n_splits=j, n_repeats = k, random_state=z)
                scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
                if np.mean(scores) > best_score:
                    best_splits = j
                    best_repeats = k
                    best_random = z
                    best_score = np.mean(scores)
    
print(best_splits)
print(best_repeats)
print(best_random)
    
    
model = LinearDiscriminantAnalysis()
cv = RepeatedStratifiedKFold(n_splits=best_splits, n_repeats = best_repeats, random_state=best_random)
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

20
1
2
Mean Accuracy: 0.853 (0.123)


In [44]:
row = [70, 1, 4, 109, 0, 2.4, 2, 3, 3]
model.fit(X, y_binary)
test = model.predict([row])
print('Predicted Class: %d' % test)

Predicted Class: 1


C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearDiscriminantAnalysis was fitted with feature names
  warnings.warn(


In [45]:
grid = dict()
grid['solver'] = ['svd', 'lsqr', 'eigen']
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(X, y)
print('Mean Accuracy: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

Mean Accuracy: 0.853
Config: {'solver': 'svd'}


In [46]:
model = LinearDiscriminantAnalysis(solver = 'lsqr')
grid = dict()
grid['shrinkage'] = np.arange(0, 1, 0.01)
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(X, y)
print('Mean Accuracy: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

Mean Accuracy: 0.853
Config: {'shrinkage': 0.0}
